# Mincut/Maxcut problem

A network is represent by a set of links shown below. Each vertex represents a person that the government can bribe (in the way mentioned in the lecture video). The price to bribe each person is also shown in the data below. 

Find the set of person to bribe such that the government gets the maximum amount of secret data. Then calculate the total price needed to bribe those people.

Solve this problem using ILP and, if you have an access to Gurobi solver, using QUBO.

In [56]:
bribe_price = {0: 12,
            1: 10,
            2: 5,
            3: 4,
            4: 10,
            5: 20,
            6: 10,
            7: 5,
            8: 4,
            9: 10,
            10: 20,
            11: 10,
            12: 5,
            13: 4,
            14: 10,
            15: 20
}

link = [(0,1),
        (0,4),
        (1,5),
        (1,7),
        (1,8),
        (1,9),
        (1,14),
        (1,15),
        (2,3),
        (3,4),
        (4,6),
        (4,7),
        (4,8),
        (4,11),
        (4,13),
        (4,15),
        (5,6),
        (5,15),
        (6,7),
        (6,8),
        (7,10),
        (7,11),
        (7,14),
        (8,10),
        (9,13),
        (10,11),
        (10,12),
        (10,13),
        (10,14),
        (10,15),
        (11,12),
        (12,15),
        (13,14),
        (14,15)
        ]

In [57]:
# using QUBO
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [58]:
# for {u, v} in Edge
# qubo = Xu(1-Xv) + Xv(1-Xu)
# qubo = Xu-XuXv + Xv-XuXv
# qubo = Xu + Xv - 2*Xu*Xv

# (binary 1^2 = 1)
#     Xu   Xv
#  Xu[1    -1]
#  Xv[-1    1]

In [78]:
N=len(bribe_price)

def construct_qubo_matrix():
    qubo_matrix=[[0] * N for _ in range(N)]  # N * N
    
    for u, v in link:
        #print(u, v)
        # Xu + Xv - 2*Xu*Xv
        '''
        qubo_matrix[u][u]+=1*bribe_price[u]
        qubo_matrix[u][v]-=1*bribe_price[u]*bribe_price[v]
        qubo_matrix[v][u]-=1*bribe_price[u]*bribe_price[v]
        qubo_matrix[v][v]+=1*bribe_price[v]
        '''
        qubo_matrix[u][u]+=1
        qubo_matrix[u][v]-=1
        qubo_matrix[v][u]-=1
        qubo_matrix[v][v]+=1

    
    return qubo_matrix

qubo=construct_qubo_matrix()
qubo=np.array(qubo)
qubo

array([[ 2, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  7,  0,  0,  0, -1,  0, -1, -1, -1,  0,  0,  0,  0, -1, -1],
       [ 0,  0,  1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  2, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0, -1,  8,  0, -1, -1, -1,  0,  0, -1,  0, -1,  0, -1],
       [ 0, -1,  0,  0,  0,  3, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0, -1, -1,  4, -1, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0, -1,  0, -1,  6,  0,  0, -1, -1,  0,  0, -1,  0],
       [ 0, -1,  0,  0, -1,  0, -1,  0,  4,  0, -1,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  7, -1, -1, -1, -1, -1],
       [ 0,  0,  0,  0, -1,  0,  0, -1,  0,  0, -1,  4, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  3,  0,  0, -1],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0, -1, -1,

In [79]:
m=gp.Model('Maxcut_Problem')
m.setParam('TimeLimit', 60)

Set parameter TimeLimit to value 60


In [80]:
x=m.addMVar(N, vtype=GRB.BINARY)
x

<MVar (16,)>
array([<gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>,
       <gurobi.Var *Awaiting Model Update*>])

In [81]:
objectives=gp.QuadExpr()
for i in range(N):
    for j in range(N):
        objectives+=x[i]*qubo[i][j]*x[j]
#m.setObjective(objectives, GRB.MINIMIZE)
#m.setObjective(objectives, GRB.MAXIMIZE)

#m.setObjective(x @ qubo @ x, GRB.MINIMIZE)
m.setObjective(x @ qubo @ x, GRB.MAXIMIZE)

m.optimize()
obj=m.getObjective()
print(obj.getValue())

m.write('mincut.sol')

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 0 rows, 16 columns and 0 nonzeros
Model fingerprint: 0x34a7a39c
Model has 50 quadratic objective terms
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 27.0000000
Presolve removed 0 rows and 4 columns
Presolve time: 0.00s
Presolved: 23 rows, 35 columns, 69 nonzeros
Variable types: 0 continuous, 35 integer (35 binary)

Root relaxation: objective 3.050000e+01, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |    

In [82]:
print(m.display())

Maximize
0.0 + [ 2.0 C0 ^ 2 + -2.0 C0 * C1 + -2.0 C0 * C4 + 7.0 C1 ^ 2 + -2.0 C1 * C5 +
-2.0 C1 * C7 + -2.0 C1 * C8 + -2.0 C1 * C9 + -2.0 C1 * C14 + -2.0 C1 * C15 + C2 ^ 2 +
-2.0 C2 * C3 + 2.0 C3 ^ 2 + -2.0 C3 * C4 + 8.0 C4 ^ 2 + -2.0 C4 * C6 + -2.0 C4 * C7 +
-2.0 C4 * C8 + -2.0 C4 * C11 + -2.0 C4 * C13 + -2.0 C4 * C15 + 3.0 C5 ^ 2 + -2.0 C5 * C6
+ -2.0 C5 * C15 + 4.0 C6 ^ 2 + -2.0 C6 * C7 + -2.0 C6 * C8 + 6.0 C7 ^ 2 + -2.0 C7 * C10
+ -2.0 C7 * C11 + -2.0 C7 * C14 + 4.0 C8 ^ 2 + -2.0 C8 * C10 + 2.0 C9 ^ 2 +
-2.0 C9 * C13 + 7.0 C10 ^ 2 + -2.0 C10 * C11 + -2.0 C10 * C12 + -2.0 C10 * C13 +
-2.0 C10 * C14 + -2.0 C10 * C15 + 4.0 C11 ^ 2 + -2.0 C11 * C12 + 3.0 C12 ^ 2 +
-2.0 C12 * C15 + 4.0 C13 ^ 2 + -2.0 C13 * C14 + 5.0 C14 ^ 2 + -2.0 C14 * C15
+ 6.0 C15 ^ 2 ]
Subject To
Binaries
['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12',
 'C13', 'C14', 'C15']
None


In [84]:
cnt_node=0
total_cost=0
for index, c in enumerate(m.getVars()):
    if c.x < 0.5:  # qubo not select but we select because it five me a correct answer?
        cnt_node+=1
        total_cost+=bribe_price[index]
        print(c.varName, c.x)
    
print("Number of person to bribe is", cnt_node)
print("Total price is", total_cost)

C1 0.0
C2 0.0
C4 0.0
C6 0.0
C10 0.0
C12 0.0
C14 0.0
Number of person to bribe is 7
Total price is 70
